In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
import tensorflow as tf

In [3]:
## Gettting the data
data = pd.read_csv('../data/heart.csv')
data['famhist'] = data['famhist'].map({"Absent":0, "Present":1})
y = pd.get_dummies(data.pop('chd')).values
feature_names = data.columns
X = data.values
print X.shape, y.shape

(462, 9) (462, 2)


In [4]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)
print np.mean(X), np.std(X)

1.17911132197e-16 1.0


In [5]:
def generate_random(orig_data, feature_names, data_size):
    '''
    Input: Data, Feature Names, Size of Random Data
    Output: Random Data
    
    Generates random dataset of size data_size
    Random dataset is generated from normal
    distribution with specified high and low values
    '''
    
    df = {}
    for i in range(len(feature_names)):
        low = np.min(orig_data[:, i])
        high = np.max(orig_data[:, i])
        df[feature_names[i]] = np.random.uniform(low=low, high=high, size=data_size)
    random_dataset = pd.DataFrame(data=df, columns=feature_names)
    
    return random_dataset

In [6]:
X_random = generate_random(X, feature_names, data_size=1000).values
print X_random.shape

(1000, 9)


In [7]:
tf.reset_default_graph()

In [8]:
with tf.variable_scope("Placeholder") as scope:
    input_tensor = tf.placeholder(dtype=tf.float32, shape=[None, 9], name="input")
    label = tf.placeholder(dtype=tf.float32, shape=[None, 2], name="label")

In [9]:
with tf.variable_scope("hidden_layer") as scope:
    w = tf.get_variable(dtype=tf.float32, name="weights", shape=[9, 128], initializer=tf.random_uniform_initializer())
    b = tf.get_variable(dtype=tf.float32, name="biases", shape=[128], initializer=tf.random_uniform_initializer())
    out = tf.nn.relu(tf.matmul(input_tensor, w) + b)

In [10]:
with tf.variable_scope("output_layer") as scope:
    w = tf.get_variable(dtype=tf.float32, name="weights", shape=[128, 2], initializer=tf.random_normal_initializer())
    b = tf.get_variable(dtype=tf.float32, name="biases", shape=[2], initializer=tf.random_normal_initializer())
    softmax = tf.nn.softmax(tf.matmul(out, w) + b)

In [11]:
with tf.variable_scope("loss") as scope:
    loss = tf.reduce_mean(-tf.reduce_sum(label * tf.log(softmax), reduction_indices=[1]))

In [12]:
with tf.variable_scope("optimizer") as scope:
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss)

In [13]:
with tf.variable_scope("accuracy") as scope:
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(softmax, 1), tf.arg_max(label, 1)), dtype=tf.float32))

In [14]:
N_EPOCHS = 5000

In [15]:
with tf.Session() as sess:
    
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(1, N_EPOCHS+1):
        
        _, l, acc = sess.run([optimizer, loss, accuracy], feed_dict={input_tensor:X, label:y})
        
        if i % 1000 == 0:
            
            print 'Epoch: {}\n Loss: {}\t Accuracy: {}'.format(i, l, acc)
            
    y_random = sess.run([softmax], feed_dict={input_tensor: X_random})

Epoch: 1000
 Loss: 0.333021968603	 Accuracy: 0.857142865658
Epoch: 2000
 Loss: 0.228172197938	 Accuracy: 0.909090936184
Epoch: 3000
 Loss: 0.164026752114	 Accuracy: 0.948051929474
Epoch: 4000
 Loss: 0.111271649599	 Accuracy: 0.969696998596
Epoch: 5000
 Loss: 0.086395509541	 Accuracy: 0.971861481667


In [18]:
y_random = y_random[0]
print y_random.shape

(1000, 2)


In [21]:
with tf.Session() as sess:
    
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(1, N_EPOCHS+1):
        
        _, l, acc = sess.run([optimizer, loss, accuracy], feed_dict={input_tensor:X_random, label:y_random})
        
        if i % 1000 == 0:
            
            print 'Epoch: {}\n Loss: {}\t Accuracy: {}'.format(i, l, acc)

Epoch: 1000
 Loss: 0.298611730337	 Accuracy: 0.887000024319
Epoch: 2000
 Loss: 0.220825895667	 Accuracy: 0.938000023365
Epoch: 3000
 Loss: 0.186767011881	 Accuracy: 0.953999996185
Epoch: 4000
 Loss: 0.170256257057	 Accuracy: 0.962999999523
Epoch: 5000
 Loss: 0.159237682819	 Accuracy: 0.971000015736


Accuracy is nice! o_O